In [14]:
#Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [15]:
#Define the path to the chrome driver
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=True)

In [160]:
#Define the scrape function

browser = init_browser()

#This section allows for the scraping of the JPL Featured Space Image website
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

html = browser.html
soup = bs(html, "html.parser")

featured_image_tag = soup.find('', class_ = 'button fancybox')['data-fancybox-href']
featured_image_url = f'https://www.jpl.nasa.gov/{featured_image_tag}'

browser.quit()

    #return listings

In [162]:
def scrape():  
    
    #Define a dictionary called scraped_data that will be returned by the function
    
    #Open a browser window
    browser = init_browser()
    
    #This section allows for the scraping of NASA's Mars Exploration website
    #Feed the URL and visit the website
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)

    #Obtain the HTML and parse it into a BeautifulSoup object.
    html = browser.html
    soup = bs(html, "html.parser")

    #Use the find function to find the most recent news title and description.
    news_title = soup.find('', class_ = 'list_text').find('a').get_text()
    news_p = soup.find('', class_ = 'article_teaser_body').get_text()
    
    #This section allows for the scraping of the JPL Featured Space Image website
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)

    html = browser.html
    soup = bs(html, "html.parser")

    featured_image_tag = soup.find('', class_ = 'button fancybox')['data-fancybox-href']
    featured_image_url = f'https://www.jpl.nasa.gov/{featured_image_tag}'
    
    browser.quit()
    
    

In [163]:
scrape()